In [1]:
from keras.models import Sequential
from keras.layers import *
from preprocess_tweets import generate_embedding_matrix
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
MAX_WORDS = 30 

In [3]:
X_train, y_train, X_test, embedding_matrix = generate_embedding_matrix()

In [4]:
X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=0.1, random_state=37)

## LSTM with GloVe embeddings

In [5]:
model = Sequential()

model.add(Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], 
                    input_length=MAX_WORDS, weights=[embedding_matrix], trainable = False))
        
model.add(LSTM(units = 100, dropout = 0.2, recurrent_dropout = 0.2))
        
model.add(Dense(1, activation = 'sigmoid'))
        
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit(X_train, np.array(y_train),
          batch_size = 512,
          epochs = 56,
          validation_data = (X_validation, y_validation))

Train on 6851 samples, validate on 762 samples
Epoch 1/56
6851/6851 [==============================] - 9s 1ms/step - loss: 0.6724 - accuracy: 0.6079 - val_loss: 0.6079 - val_accuracy: 0.7192
Epoch 2/56
6851/6851 [==============================] - 7s 955us/step - loss: 0.5812 - accuracy: 0.7200 - val_loss: 0.5221 - val_accuracy: 0.7612
Epoch 3/56
6851/6851 [==============================] - 7s 1ms/step - loss: 0.5440 - accuracy: 0.7446 - val_loss: 0.4897 - val_accuracy: 0.7743
Epoch 4/56
6851/6851 [==============================] - 8s 1ms/step - loss: 0.5275 - accuracy: 0.7535 - val_loss: 0.4732 - val_accuracy: 0.7848
Epoch 5/56
6851/6851 [==============================] - 8s 1ms/step - loss: 0.5189 - accuracy: 0.7605 - val_loss: 0.4636 - val_accuracy: 0.7900
Epoch 6/56
6851/6851 [==============================] - 10s 1ms/step - loss: 0.5179 - accuracy: 0.7543 - val_loss: 0.4639 - val_accuracy: 0.7940
Epoch 7/56
6851/6851 [==============================] - 10s 1ms/step - loss: 0.5121 - 

#### Create a submission file

In [7]:
test_predict = model.predict_classes(X_test)[:, 0]

In [8]:
df_test = pd.read_csv("../data/test.csv")
ids = df_test['id'].to_numpy()

In [9]:
pd.DataFrame({'id': ids, 'target': test_predict}).to_csv('../data/2. Submission_LSTM.csv', index=False)

### Bidirectional LSTM

In [10]:
model = Sequential()

model.add(Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], 
                    input_length = MAX_WORDS, weights=[embedding_matrix], trainable = False))

model.add(Bidirectional(LSTM(units = 100, dropout = 0.2, recurrent_dropout = 0.2)))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [11]:
model.fit(X_train, y_train,
          batch_size = 512,
          epochs = 25,
          validation_data = (X_validation, y_validation))

Train on 6851 samples, validate on 762 samples
Epoch 1/25
6851/6851 [==============================] - 14s 2ms/step - loss: 0.6520 - accuracy: 0.6275 - val_loss: 0.5905 - val_accuracy: 0.7139
Epoch 2/25
6851/6851 [==============================] - 11s 2ms/step - loss: 0.5504 - accuracy: 0.7316 - val_loss: 0.4966 - val_accuracy: 0.7743
Epoch 3/25
6851/6851 [==============================] - 11s 2ms/step - loss: 0.5070 - accuracy: 0.7644 - val_loss: 0.4757 - val_accuracy: 0.7913
Epoch 4/25
6851/6851 [==============================] - 11s 2ms/step - loss: 0.4917 - accuracy: 0.7711 - val_loss: 0.4706 - val_accuracy: 0.7966
Epoch 5/25
6851/6851 [==============================] - 11s 2ms/step - loss: 0.4902 - accuracy: 0.7697 - val_loss: 0.4547 - val_accuracy: 0.8018
Epoch 6/25
6851/6851 [==============================] - 11s 2ms/step - loss: 0.4822 - accuracy: 0.7794 - val_loss: 0.4549 - val_accuracy: 0.7979
Epoch 7/25
6851/6851 [==============================] - 11s 2ms/step - loss: 0.4855

#### Create a submission file

In [12]:
test_predict = model.predict_classes(X_test)[:, 0]

In [13]:
df_test = pd.read_csv("../data/test.csv")
ids = df_test['id'].to_numpy()

In [14]:
pd.DataFrame({'id': ids, 'target': test_predict}).to_csv('../data/3. Submission_Bidirectional_LSTM.csv', index=False)

### CNN LSTM

In [15]:
model = Sequential()

model.add(Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], 
                    input_length = MAX_WORDS, weights=[embedding_matrix], trainable = False))

model.add(Conv1D(filters = 128, kernel_size = 4, padding = 'valid', activation = 'relu'))

model.add(MaxPooling1D(pool_size = 2))

model.add(LSTM(units = 100, dropout = 0.2, recurrent_dropout = 0.1))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
model.fit(X_train, y_train,
          batch_size = 512,
          epochs = 20,
          validation_data = (X_validation, y_validation))

Train on 6851 samples, validate on 762 samples
Epoch 1/20
6851/6851 [==============================] - 10s 1ms/step - loss: 0.6521 - accuracy: 0.6358 - val_loss: 0.5705 - val_accuracy: 0.7244
Epoch 2/20
6851/6851 [==============================] - 6s 869us/step - loss: 0.5483 - accuracy: 0.7387 - val_loss: 0.4966 - val_accuracy: 0.7769
Epoch 3/20
6851/6851 [==============================] - 5s 740us/step - loss: 0.4993 - accuracy: 0.7732 - val_loss: 0.4726 - val_accuracy: 0.7756
Epoch 4/20
6851/6851 [==============================] - 5s 735us/step - loss: 0.4852 - accuracy: 0.7784 - val_loss: 0.4545 - val_accuracy: 0.7808
Epoch 5/20
6851/6851 [==============================] - 6s 925us/step - loss: 0.4737 - accuracy: 0.7847 - val_loss: 0.4472 - val_accuracy: 0.7992
Epoch 6/20
6851/6851 [==============================] - 7s 1ms/step - loss: 0.4609 - accuracy: 0.7889 - val_loss: 0.4433 - val_accuracy: 0.7900
Epoch 7/20
6851/6851 [==============================] - 7s 1ms/step - loss: 0.45

#### Create submission csv

In [17]:
test_predict = model.predict_classes(X_test)[:, 0]

In [18]:
df_test = pd.read_csv("../data/test.csv")
ids = df_test['id'].to_numpy()

In [20]:
pd.DataFrame({'id': ids, 'target': test_predict}).to_csv('../data/4. Submission_CNN_LSTM.csv', index=False)